In [1]:
from __future__ import division
from __future__ import print_function
import os, sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import argparse
import time
import random
import numpy as np
import scipy.sparse as sp
import torch

SEED = 4
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

from torch import optim
import torch.nn.functional as F
from model import ROD_lp
from optimizer import loss_function
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn import metrics
import copy

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--num_hops', type=int, default=5)
parser.add_argument('--epochs', type=int, default=400)
parser.add_argument('--dims', type=int, default=[1024])
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=int, default=500)
parser.add_argument('--dataset', type=str, default='cora')
parser.add_argument('--device', type=int, default=0)
parser.add_argument('--upd', type=int, default=10)
args = parser.parse_args(args=[])
print("Using {} dataset".format(args.dataset))

Using cora dataset


### 1. Data Fetching

In [3]:
device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
adj, features, true_labels, idx_train, idx_val, idx_test = load_data(args.dataset)
n_nodes, feat_dim = features.shape
dims = [feat_dim] + args.dims

adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj.eliminate_zeros()
adj_orig = adj

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train
n = adj.shape[0]

adj_normalized = preprocess_graph(adj, norm='sym', renorm=True)
features = sp.csr_matrix(features).toarray()
    
feature_list = [features]
for i in range(args.num_hops):
    feature_list.append(adj_normalized.dot(feature_list[-1]))
input_feature = [torch.FloatTensor(feat).to(device) for feat in feature_list] 

adj_1st = (adj + sp.eye(n)).toarray()

adj_label = torch.FloatTensor(adj_1st)

neg_num = pos_num = adj_label.sum().long()

model = ROD_lp(dims, args.num_hops)

optimizer = optim.Adam(model.parameters(), lr=args.lr)

### 2. Data Preprocessing

In [4]:
sm_sim_mx_list = []
for i in range(args.num_hops+1):
    cur_feat = F.normalize(input_feature[i].cpu().data)
    sm_sim_mx_list.append(torch.mm(cur_feat, cur_feat.t()).reshape([-1,]))

adj_label = adj_label.reshape([-1, ])

model = model.to(device)
adj_label = adj_label.to(device)

pos_inds_list = []
neg_inds_list = []
for i in range(args.num_hops+1):
    pos_inds_list.append(np.argpartition(-sm_sim_mx_list[i], pos_num)[:pos_num])
    neg_inds_list.append(np.argpartition(sm_sim_mx_list[i], pos_num*200)[:pos_num*200])

pos_inds_ensemble = np.concatenate(pos_inds_list, axis=0)
neg_inds_ensemble = np.concatenate(neg_inds_list, axis=0)
    
length_ensemble = len(pos_inds_ensemble)
length_ensemble_neg = len(neg_inds_ensemble)
    
length = len(pos_inds_list[0])
length_neg = len(neg_inds_list[0])

pos_inds_ensemble_cuda = torch.LongTensor(pos_inds_ensemble).to(device)
pos_inds_cuda_list = [torch.LongTensor(pos_inds).to(device) for pos_inds in pos_inds_list]

batch_size = args.batch_size

### 3. Link Prediction

In [5]:
best_lp = 0.
best_emb_list = []
tqdm.write('Start Training...')
for epoch in tqdm(range(args.epochs)):
    model.train()
    optimizer.zero_grad()

    all_time_per_epoch = 0.
    t = time.time()
    z_list = model(input_feature)

    start, end = 0, batch_size
    start_ensemble, end_ensemble = 0, batch_size*args.num_hops
    batch_num = 0
    loss = 0.

    ran_head = np.random.randint(0, length_neg - length - 1)
    sampled_neg_list = []
    for i in range(args.num_hops+1):
        sampled_neg_list.append(torch.LongTensor(neg_inds_list[i][np.arange(ran_head, ran_head+length)]).to(device))
   
    ran_head_0 = np.random.randint(0, length_ensemble_neg - length_ensemble - 1)
    sample_neg_ensemble = torch.LongTensor(neg_inds_ensemble[np.arange(ran_head_0, ran_head_0+length_ensemble)]).cuda()

    while end_ensemble <= length_ensemble:
        sample_ensemble_inds = torch.cat((pos_inds_ensemble_cuda[start_ensemble:end_ensemble], sample_neg_ensemble[start_ensemble:end_ensemble]), 0)
        sample_ensemble_inds = sample_ensemble_inds.cuda()
        xind = sample_ensemble_inds // n_nodes
        yind = sample_ensemble_inds % n_nodes
        batch_label_original = torch.index_select(adj_label, 0, sample_ensemble_inds)

        batch_pred_ensemble_list = []
        for i in range(args.num_hops+1):
            sampled_inds = torch.cat((pos_inds_cuda_list[i][start:end], sampled_neg_list[i][start:end]), 0).to(device)
            xind = sampled_inds // n_nodes
            yind = sampled_inds % n_nodes
            zx_ensemble = torch.index_select(z_list[i], 0, xind)
            zy_ensemble = torch.index_select(z_list[i], 0, yind)
            zx = torch.index_select(z_list[i], 0, xind)
            zy = torch.index_select(z_list[i], 0, yind)
            batch_label = torch.cat((torch.ones(end-start), torch.zeros(end-start))).to(device)
            batch_label_original = torch.index_select(adj_label, 0, sampled_inds)
            batch_pred = (zx * zy).sum(1)
            batch_pred_ensemble = (zx_ensemble * zy_ensemble).sum(1)
            batch_pred_ensemble_list.append(batch_pred_ensemble)
            weight = torch.cat((batch_pred[:batch_size], 1-batch_pred[batch_size:]), 0).data
            loss += loss_function(adj_preds=batch_pred, adj_labels=batch_label_original)
            sm_sim_mx = sm_sim_mx_list[i].to(device)
            batch_label_soft = torch.index_select(sm_sim_mx, 0, sampled_inds)
            loss += 0.2*loss_function(adj_preds=batch_pred, adj_labels=batch_label, weight=weight)
            loss += 0.2*F.mse_loss(batch_pred, batch_label_soft)
        
        attention_scores = [torch.sigmoid(model.lr_att2(batch_pred.view(-1,1))).view(batch_pred.shape[0], 1) for batch_pred in batch_pred_ensemble_list]
        W = torch.cat(attention_scores, dim=1)
        W = F.softmax(W, 1)

        pred_ensemble = torch.mul(batch_pred_ensemble_list[0], W[:, 0])
        for i in range(1, args.num_hops+1):
            pred_ensemble += torch.mul(batch_pred_ensemble_list[i], W[:, i])

        #teacher loss
        for i in range(args.num_hops+1):
            loss += 0.1*F.kl_div(F.log_softmax(batch_pred_ensemble_list[i], dim=-1), F.softmax(pred_ensemble, dim=-1), reduction='mean')
            loss += 0.1*loss_function(adj_preds=batch_pred_ensemble_list[i], adj_labels=pred_ensemble)
            loss += loss_function(adj_preds=batch_pred_ensemble_list[i], adj_labels=batch_label_original)
     
        start_ensemble = end_ensemble
        start = end
        if end_ensemble < length_ensemble <= end_ensemble + (args.num_hops+1)*batch_size:
            break
        else:
            end += batch_size
            end_ensemble += (args.num_hops+1)*batch_size
        
    loss.backward()
    cur_loss = loss.item()
    optimizer.step()

    if (epoch + 1) % args.upd == 0:
        model.eval()
        z_list = model(input_feature)
        z_list = [zz.cpu().data.numpy() for zz in z_list]
    
        val_auc, val_ap = get_roc_score_ensemble(z_list, adj_orig, val_edges, val_edges_false)
        if val_auc + val_ap >= best_lp:
            best_lp = val_auc + val_ap
            best_emb_list = [z_list[0]]
            for i in range(1, len(z_list)):
                best_emb_list.append(z_list[i])
        tqdm.write("Epoch: {}, train_loss_gae={:.5f}, time={:.5f}".format(
            epoch + 1, cur_loss, time.time() - t))

tqdm.write("Optimization Finished!")
auc_score, ap_score = get_roc_score_ensemble(best_emb_list, adj_orig, test_edges, test_edges_false)
tqdm.write('Test AUC score: ' + str(auc_score))
tqdm.write('Test AP score: ' + str(ap_score))


  0%|          | 0/400 [00:00<?, ?it/s]

Start Training...


  2%|▎         | 10/400 [00:05<03:52,  1.68it/s]

Epoch: 10, train_loss_gae=282.18314, time=0.92969


  5%|▌         | 20/400 [00:10<04:16,  1.48it/s]

...

Epoch: 380, train_loss_gae=231.42209, time=0.92595


 98%|█████████▊| 390/400 [03:13<00:05,  1.69it/s]

Epoch: 390, train_loss_gae=231.00861, time=0.92533


100%|██████████| 400/400 [03:17<00:00,  2.02it/s]


Epoch: 400, train_loss_gae=230.78877, time=0.90777
Optimization Finished!
Test AUC score: 0.9599717710429951
Test AP score: 0.9640584566332792
